# Продвинутое машинное обучение: ДЗ 2
## *Латыпов Ильяс, группа DS-22*
### Часть 1.

Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:
   - взять в тренировочный набор турниры с dateStart из 2019 года; 
   - в тестовый — турниры с dateStart из 2020 года.


In [85]:
import numpy as np
import pandas as pd

import zipfile
import pickle

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()


# from collections import defaultdict
from scipy.sparse import csr_matrix
from sklearn.linear_model import LogisticRegression, LinearRegression
# from scipy.stats import spearmanr, kendalltau

# Загрузка и обработка данных

In [2]:
myzip = zipfile.ZipFile("D:\\ID\\Course\\MADE\\2.Adv_ML\\HW\\chgk.zip")
print("Содержание zip файла:")
for file_name in myzip.namelist():
    print(file_name)

Содержание zip файла:
players.pkl
results.pkl
tournaments.pkl


### Загрузка tournaments.pkl

In [3]:
data_tournament = pd.DataFrame(pickle.load(myzip.open('tournaments.pkl', 'r'))).T
data_tournament["year"] = data_tournament['dateStart'].apply(lambda x: int(x[:4]) if x is not None else None)
data_tournament['sum_questionQty'] = data_tournament['questionQty'].apply(lambda x: sum(x.values()) if x is not None else 0)
train_tournament = data_tournament[data_tournament['year']==2019]
test_tournament = data_tournament[data_tournament['year']==2020]
train_tournament = train_tournament[["id", "year", "sum_questionQty"]]
test_tournament = test_tournament[["id", "year", "sum_questionQty"]]
print("Количество соревнований в 2019 в tournaments.pkl:", train_tournament.shape)
print("Количество соревнований в 2020 в tournaments.pkl:", test_tournament.shape)

Количество соревнований в 2019 в tournaments.pkl: (687, 3)
Количество соревнований в 2020 в tournaments.pkl: (418, 3)


### Загрузка players.pkl

In [4]:
data_players = pd.DataFrame(pickle.load(myzip.open('players.pkl', 'r'))).T
print("Общее количество участников в players.pkl:", data_players.shape)
display(data_players.sample(2))

Общее количество участников в players.pkl: (204063, 4)


,id,name,patronymic,surname
217116,217116,Владимир,Александрович,Варивода
152643,152643,Ульяна,Юрьевна,Кобылец


### Загрузка results.pkl

In [25]:
%%time
results = pickle.load(myzip.open('results.pkl', 'r'))
tournament_test_list = test_tournament["id"].to_list() 
tournament_train_list = train_tournament["id"].to_list()

data_result = pd.DataFrame([])
for tour in (tournament_train_list + tournament_test_list):
    if tour not in results.keys():
        print(f"Не найдено соревнование № {tour} в results.pkl")
        break
    cur_tour_df = pd.DataFrame(results[tour])
    cur_tour_df["tournament"] = tour
    if tour in tournament_train_list:
        cur_tour_df["year"] = 2019
    else:
        cur_tour_df["year"] = 2020
    data_result = pd.concat([data_result, cur_tour_df])
    
data_result["teamMembers_empty"] = data_result["teamMembers"].apply(lambda x: not x) 
data_result = data_result[(~data_result["mask"].isna()) & (~data_result["teamMembers_empty"])]
data_result.drop(columns=["teamMembers_empty"], inplace=True)

data_result["players_id"] = data_result["teamMembers"].apply(lambda x: list(map(lambda y: y['player']['id'], x)))
data_result["players_number"] = data_result["teamMembers"].apply(lambda x: len(list(map(lambda y: y['player']['id'], x))))
data_result["players_name"] = data_result["teamMembers"].apply(lambda x: list(map(lambda y: y['player']['surname'] + " " +
                                                               y['player']['name'] + " " + y['player']['patronymic'], x)))

data_result["team_id"] = data_result["team"].apply(lambda x: x['id'])
data_result = data_result[["tournament", "year", "team_id", "mask", "questionsTotal", 
                           "players_id", "players_name", "players_number"]]

# data_result = data_result[~data_result['mask'].apply(lambda x: "X" in x)]
data_result['mask'] = data_result['mask'].apply(lambda x: x.replace("?","0").replace("-","0").replace("X",""))
data_result["mask_length"] = data_result["mask"].apply(len)

temp = pd.DataFrame(data_result.groupby(by = "tournament")["mask_length"].max())
temp.rename(columns = {"mask_length" : "tour_mask_length"}, inplace=True)
data_result = pd.merge(data_result, temp, on='tournament', how="left")
data_result = data_result[data_result["mask_length"] == data_result["tour_mask_length"]]
data_result.drop(columns=["tour_mask_length"], inplace=True)

# temp = data_tournament[(data_tournament['year']==2019) | (data_tournament['year']==2020)][["id", "sum_questionQty"]]
# temp.rename(columns = {"id" : "tournament"}, inplace=True)
# data_result = pd.merge(data_result, temp, on='tournament', how="left")
# data_result = data_result[data_result["mask_length"]==data_result["sum_questionQty"]]

data_result_ext = data_result.loc[data_result.index.repeat(data_result.players_number)]
data_result_ext = data_result_ext.reset_index()
data_result_ext.drop(columns = ["index"], inplace=True)
data_result_ext = data_result_ext.reset_index()
data_result_ext["list_ind"] = data_result_ext["index"] % data_result_ext["players_number"]
data_result_ext["player_id"] = data_result_ext.apply(lambda x: x["players_id"][x["list_ind"]], axis=1)
data_result_ext["player_name"] = data_result_ext.apply(lambda x: x["players_name"][x["list_ind"]], axis=1)
data_result_ext.drop(columns=["players_id","players_name", "players_number", "list_ind", "index"], inplace=True)
data_result_ext.rename(columns = {"tournament" : "tournament_id"}, inplace=True)

train_result = data_result_ext[data_result_ext["year"] == 2019]
test_result = data_result_ext[data_result_ext["year"] == 2020]
test_result = test_result[test_result["player_id"].isin(set(train_result.player_id.to_list()))]
train_result.to_pickle('result_train.pkl')
test_result.to_pickle('result_test.pkl')

print("Сумма участников во всех турнирах 2019г", train_result.shape)
print("Сумма участников во всех турнирах 2020г:", test_result.shape)
display(train_result.head(2))

Сумма участников во всех турнирах 2019г (414774, 8)
Сумма участников во всех турнирах 2020г: (99757, 8)


,tournament_id,year,team_id,mask,questionsTotal,mask_length,player_id,player_name
0,4772,2019,45556,111111111011111110111111111100010010,28.0,36,6212,Выменец Юрий Яковлевич
1,4772,2019,45556,111111111011111110111111111100010010,28.0,36,18332,Либер Александр Витальевич


Wall time: 1min 40s


In [26]:
print("Выборка соревнований для обучения:", data_result[data_result["year"] == 2019].tournament.nunique())
print("Выборка соревнований для теста:", data_result[data_result["year"] == 2020].tournament.nunique())

Выборка соревнований для обучения: 675
Выборка соревнований для теста: 173


In [113]:
train_result = pd.read_pickle('result_train.pkl')
test_result = pd.read_pickle('result_test.pkl')
train_result.shape, test_result.shape

((414774, 8), (99757, 8))

In [114]:
players_train_ids = set(train_result.player_id.to_list())
players_test_ids = set(test_result.player_id.to_list())
print("Количество участников в 2019г: ", len(players_train_ids))
print("Количество участников в 2020г: ", len(players_test_ids))

Количество участников в 2019г:  57424
Количество участников в 2020г:  22946


### Часть 2.

Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
  - повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
  - в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
  - для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.


In [111]:
len(players_train_ids)

57424

In [66]:
enum_players_ids = {k: i for i, k in enumerate(players_train_ids)}
tournament_new_id_map = {k: i for i, k in enumerate(set(train_result["tournament_id].to_list()))}

train_result['player_new_id'] = train_result['player_id'].map(enum_players_ids)
train_result['tournament_new_id'] = train_result['tournament_id'].map(tournament_new_id_map)
test_result['player_new_id'] = test_result['player_id'].map(enum_players_ids)

tournaments_max_questions = train_result.groupby('tournament_new_id')['mask_length'].max()
tournament_question_start_id_dict = dict(tournaments_max_questions.cumsum() - 36)

In [108]:
train_result.shape[0]

414774

In [74]:
%%time
X, y = [], []
for mask, playes_id, tournament_id  in zip(train_result['mask'], train_result['player_new_id'], train_result['tournament_new_id']):
    for i, target in enumerate(mask):
        assert target in ['0', '1'], f'Got incorrect mask value: {target} (tournament_id={tid}, player_id={pid})'
        if target == '1':
            target = 1
        else:
            target = 0
        question_id = tournament_question_start_id_dict[tournament_id] + i
        X.append([question_id, playes_id])
        y.append(target)
train_X, train_y = np.array(X), np.array(y)

Wall time: 44.2 s


In [77]:
train_X.shape, train_y.shape

((17753544, 2), (17753544,))

In [109]:
for ind in range(train_result.shape[0]):
    display(train_result.iloc[ind])
    break

tournament_id                                        4772
year                                                 2019
team_id                                             45556
mask                 111111111011111110111111111100010010
questionsTotal                                       28.0
mask_length                                            36
player_id                                            6212
player_name                        Выменец Юрий Яковлевич
player_new_id                                           0
tournament_new_id                                       0
Name: 0, dtype: object

In [82]:
question_max_token = tournaments_max_questions.cumsum().max()
question_max_token

33303

In [89]:
def to_csr(X):
    x_rows = np.repeat(np.arange(X.shape[0]), 2)
    x_cols = X.copy()
    x_cols[:,1] += question_max_token + 1
    x_cols = x_cols.ravel()
    x_values = np.ones(X.shape[0] * 2, dtype=np.uint8)
    csr_x = csr_matrix((x_values, (x_rows, x_cols)), dtype=np.uint8)
    return csr_x

train_X_csr = to_csr(train_X)

In [92]:
%%time
model = LogisticRegression(max_iter=300, solver="lbfgs", fit_intercept=False)
model.fit(train_X_csr, train_y)
print("Mean accuracy:", model.score(train_X_csr, train_y))

Wall time: 2min 32s


LogisticRegression(fit_intercept=False, max_iter=300)

In [94]:
player_weights = model.coef_[0, question_max_token + 1:]
players_rating = {i:rating for i, rating in enumerate(player_weights)}

In [97]:
player_weights.shape, len(players_rating)

(57424,)

In [103]:
players_rating_df = train_result[['player_name', 'player_new_id']].drop_duplicates(subset=['player_new_id'])
players_rating_df['rating'] = players_rating_df['player_new_id'].map(players_rating)
players_rating_df = players_rating_df.sort_values(by='rating', ascending=False)
players_rating_df = players_rating_df.reset_index(drop=True)
players_rating_df.head(15)

,player_name,player_new_id,rating
0,Смирнов Сергей Анатольевич,131,3.467645
1,Бурак Ирина Викторовна,122,3.132473
2,Вакуленко Сергей Владимирович,168,2.718609
3,Руссо Максим Михайлович,8223,2.659385
4,Брутер Александра Владимировна,5957,2.547613
5,Семушин Иван Николаевич,1208,2.522397
6,Савченков Михаил Владимирович,1212,2.416838
7,Выменец Юрий Яковлевич,0,2.383318
8,Спешков Сергей Леонидович,8244,2.341007
9,Либер Александр Витальевич,1,2.332709


In [106]:
players_rating_df[players_rating_df['player_name'] == 'Николенко Сергей Игоревич']

,player_name,player_new_id,rating
15528,Николенко Сергей Игоревич,3,-0.338739


In [29]:
class PlayerTokenizer(object):
    """Назначает каждому игроку уникальное натуральное число.
    Числа идут подряд начиная с нуля."""
    def __init__(self):
        self.player2token = {}
        self.token2player = {}
    
    def fit(self, results):
        player_ids = results.player_id.unique()
        self.player2token = {id: i for i, id in enumerate(player_ids)}
        self.token2player = {i: id for i, id in enumerate(player_ids)}
        return self
    
    def get_token(self, player_id):
        return self.player2token[player_id] if player_id in self.player2token else None
    
    def get_player(self, token):
        return self.token2player[token] if token in self.token2player else None

In [30]:
pt = PlayerTokenizer().fit(train_result)
(158635), pt.get_player(pt.get_token(158635))

(158635, 158635)

In [31]:
class QuestionTokenizer(object):
    """Назначает каждому вопросу уникальное натуральное число.
    Числа идут подряд начиная с нуля."""
    def __init__(self):
        self.question2token = {}
        self.token2question = {}
        self.max_token = None
    
    def fit(self, results):
        results = results.drop_duplicates('tournament')
        last_id = 0
        for tournament_id, mask_len in zip(results.tournament_id, results.mask_len):
            tournament_tokens = np.arange(last_id, last_id + mask_len)
            for i, tokens in enumerate(tournament_tokens):
                self.token2question[tokens] = (tournament_id, i)
            last_id += mask_len
            self.question2token[tournament_id] = tournament_tokens
        self.max_token = last_id - 1
        return self
    
    def get_token(self, tournament_id, question_index):
        return self.question2token[tournament_id][question_index]
    
    def get_question(self, token):
        return self.token2question[token]

In [32]:
qt = QuestionTokenizer().fit(train_result)

KeyError: Index(['tournament'], dtype='object')